# NX-MIMOSA Intelligence Pipeline## Platform ID + ECM Detection + Intent PredictionThis notebook demonstrates the full intelligence assessment:1. Platform classification from kinematics (31 types)2. ECM detection (noise, deception, DRFM, chaff)3. Intent prediction (16 behavioral types)4. Composite threat scoring

In [ ]:
import numpy as npfrom python.nx_mimosa_intelligence import (    IntelligencePipeline, PlatformClassifier, ECMDetector, IntentPredictor,    ThreatLevel, ECMStatus, IntentType,)

## Scenario: Fighter Ingress Under Jamming

In [ ]:
intel = IntelligencePipeline()np.random.seed(42)print("Scan | Platform         | ECM        | Intent        | Threat | Range")print("-----|-----------------|------------|---------------|--------|------")for scan in range(40):    # Fighter approaching at 450 m/s, descending, pulling g's after scan 25    speed = 450    alt = 8000 - scan * 100  # Descending    g = 2.0 + (scan > 25) * 5.0  # High-g after scan 25    rng = 120000 - scan * 3000    heading = 0.0 + (scan > 30) * 0.3  # Heading change at scan 30        # ECM: jamming starts at scan 15    snr = 22.0 if scan < 15 else 8.0    nis = 1.5 if scan < 15 else 12.0        report = intel.assess(        track_id=1,        speed_ms=speed, altitude_m=max(alt, 200),        heading_rad=heading, g_load=g,        range_to_asset_m=max(rng, 1000),        snr_db=snr, rcs=5.0, doppler_hz=450*3.0, nis=nis,    )        if scan % 5 == 0:        print(f" {scan:3d}  | {report.platform.platform_type:16s} | "              f"{report.ecm.status.value:10s} | "              f"{report.intent.primary_intent.value:13s} | "              f"{report.composite_threat:.2f}   | {max(rng,1000)/1000:.0f}km")

## Platform Classification Gallery

In [ ]:
scenarios = [    ("Commercial Airliner", 240, 11000, 1.1),    ("Fighter Jet (Mach 1.5)", 500, 10000, 7.0),    ("Helicopter", 60, 500, 1.5),    ("Cruise Missile", 280, 50, 4.0),    ("Ballistic Missile", 3500, 200000, 1.5),    ("UAV (Tactical)", 40, 3000, 2.0),    ("SAM Launch", 800, 15000, 25.0),    ("Business Jet", 230, 12000, 2.0),]print(f"{'Scenario':<30s} {'Platform Type':<20s} {'Class':<15s} {'Threat':<10s} {'Conf':<6s}")print("-" * 85)for name, spd, alt, g in scenarios:    clf = PlatformClassifier()    for _ in range(25):        result = clf.update(spd, alt, g)    print(f"{name:<30s} {result.platform_type:<20s} {result.coarse_class:<15s} "          f"{result.threat_level.name:<10s} {result.confidence:.2f}")

## ECM Detection Demo

In [ ]:
ecm = ECMDetector()# Phase 1: Clean (20 scans)for _ in range(20):    r = ecm.update(snr_db=25.0, rcs=1.0, doppler_hz=100, nis=1.0)print(f"Phase 1 (Clean):    {r.status.value}")# Phase 2: Noise Jamming (15 scans)for _ in range(15):    r = ecm.update(snr_db=5.0, rcs=1.0, doppler_hz=100, nis=8.0)print(f"Phase 2 (Jamming):  {r.status.value}, Q scale={r.q_scale_factor}×")print(f"  SNR drop: {r.indicators['snr_drop_db']:.1f} dB")